<a href="https://colab.research.google.com/github/Carlinpe/Algorithm-to-win-a-sports-bet-with-python/blob/main/Actividad03/Actividad03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Actividad 03
```
Curso     : Minería de Datos 2021-2
Docente   : Carlos Fernando Montoya Cubas
Autor     : Carlos Enrique Quispe Chambilla
Fecha     : 24/11/2021
Lugar     : Cusco, Perú, 2021.
Proposito : Implementar el algoritmo de Apriori
```

##**Librerias**






In [8]:
import numpy as np
import pandas as pd
import collections
from collections import defaultdict

## **1. Implementar las siguientes funciones principales**
*   **get_frequent_itemsets(playlists, min_support):** Recibe la estructura de datos que contiene a las playlists y retorna una estructura con los itemsets fre-cuentes, bajo un umbral mínimo de soporte.

*   **generate_association_rules(frequent_itemsets, confidence, lift):** Recibe los itemsets frecuentes generados por la función anterior y retorna las reglas de asociación. Se le puede entregar umbrales de confianza o lift para las reglas que se retornarán. Por ejemplo, si se llama esta función con los ar- gumentos confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2.

#### **Funcion: get_frequent_itemsets(playlists, min_support)**

In [9]:
# Funcion para estructurar los datos con un umbral minimo de aceptacion
def get_frequent_itemsets(playlists, min_support):
  # Listamos los datos
  playlists = list(playlists.item().values())
  playlists = [set(playlist) for playlist in playlists]
  canciones = [item for sublist in playlists for item in sublist]
  # Determinamos la frecuencia de cada cancion
  contador_canciones = pd.Series(data=canciones).value_counts().to_dict()
  # Se crea un diccionario de canciones
  canciones_playlists = collections.defaultdict(set)
  # Determinamos un valor para cada playlist
  for index, playlist in enumerate(playlists):
      for cancion in playlist:
          canciones_playlists[cancion].add(index)
  canciones_playlists = canciones_playlists
  # Determinamos las canciones mas rankeadas en cada playlist
  contador = {cancion: veces for cancion, veces in contador_canciones.items() if veces / len(playlists) >= min_support}
  itemsets = [{cancion} for cancion in contador.keys()]
  # Obtenemos los itemsets mas frecuentes
  itemsets_frecuentes = {}
  itemsets_frecuentes[1] = sorted(contador.items(), key=lambda x: x[1], reverse=True)
  itemsets_frecuentes_tamaño = []
  # Inicializamos contador en 2
  k = 2
  actual = itemsets  
  while len(actual) != 0:
    # Generamos itemsets
    combinaciones = set() 
    m = k - 2
    for candidato in actual: 
      candidato = list(candidato)
      for aux in actual:
        aux = list(aux)
        union = True
        for i in range(k - 2):
          if candidato[i] != aux[i]:
              union = False
              break
        if not union:
          continue
        if candidato[k - 2] < aux[k - 2]:
          c = candidato + [aux[k - 2]]
          c = frozenset(sorted(c))
          combinaciones.add(c)
    # Determinamos que el itemset cumpla cumpla con el umbral minimo de soporte
    contadorDecombinaciones = {}   
    playlist_tamanio = len(playlists)   
    for candidato in combinaciones:
        playlists_inter = []
        for song in candidato:
          playlists_inter.append(canciones_playlists[song])
        contadorDecombinaciones[candidato] = len(set.intersection(*playlists_inter))
    contador_libreria = {subset: veces for subset, veces in contadorDecombinaciones.items() if veces / playlist_tamanio >= min_support} 
    libreria = contador_libreria.keys() 
    itemsets_frecuentes_tamaño.extend(libreria)
    itemsets_frecuentes[k] = sorted(contador_libreria.items(), key=lambda x: x[1], reverse=True)
    k += 1
    actual = libreria
  # Generamos itemsets usando DataFrame
  itemsets_dataframe = pd.DataFrame([item for sublist in itemsets_frecuentes.values() for item in sublist]).round(3)
  itemsets_dataframe.columns = ["itemset", "contador_support"]
  itemsets_dataframe["support"] = itemsets_dataframe["contador_support"] / len(playlists)
  # La funcion retornara un vector con 4 datos
  extra_data = []
  # En la primera posicion estaran los itemsets mas frecuentes.
  extra_data.append(itemsets_dataframe)
  # En la segunda posicion estaran los datos de la playlist para un posterior uso.
  extra_data.append(playlists)
  # En la tercera posicion estara el registro de las canciones que estan en las playlists.
  extra_data.append(canciones_playlists)
  # En la cuarta posicion estaran el registro de los itemsets largos para un posterior uso.
  extra_data.append(itemsets_frecuentes_tamaño)
  return extra_data


#### **Funcion: generate_association_rules(frequent_itemsets, confidence, lift)**

In [16]:
#Funcion para generar las reglas de asociacion
def generate_association_rules(frequent_itemsets, confidence, lift):
  reglas = [] 
  for itemset in frequent_itemsets[3]:
    #Se forman todas las combinaciones posibles de X -> Y, donde X ⊂ I y Y = I - X
    itemset_contador = calcular_interseccion(itemset,frequent_itemsets[2])
    itemset_support = itemset_contador / len(frequent_itemsets[0])
    for i in range(1, len(itemset) + 1):
        for xSet in combinaciones(itemset, i):
            xSet = set(xSet)
            ySet = set(itemset) - xSet
            xSupport = calcular_interseccion(xSet,frequent_itemsets[2]) / len(frequent_itemsets[0])
            xySupport = calcular_interseccion(xSet.union(ySet),frequent_itemsets[2]) / len(frequent_itemsets[0])
            rule_confidence = xySupport / xSupport
            if len(xSet) > 0 and len(ySet) > 0:
                ySupport = calcular_interseccion(ySet,frequent_itemsets[2]) / len(frequent_itemsets[0])
                rule_lift = xySupport / (xSupport * ySupport)
                reglas.append((xSet, ySet, rule_confidence, xySupport, rule_lift))
  #Generar dataframe de las reglas de asociacion
  reglas_dataframe = pd.DataFrame(data=reglas, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  reglas_dataframe["X"] = list(map(tuple, reglas_dataframe["X"]))
  reglas_dataframe["Y"] = list(map(tuple, reglas_dataframe["Y"]))

  #Hallar las reglas de asociacion de 10 elementos para la confianza
  order_by="confidence"
  n=10
  confidence_df = reglas_dataframe.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion de 10 elementos para lift
  order_by="lift"
  n=10
  lift_df2 = reglas_dataframe.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion que cumplan con la condicion de confianza y lift
  reglas_dataframe["len_itemset"] = reglas_dataframe.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  reglas_dataframe=reglas_dataframe.sort_values(by="len_itemset", ascending=False)
  both_rules = reglas_dataframe[(reglas_dataframe["confidence"] >= confidence) & (reglas_dataframe["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules


### **1.1 Implementar las siguientes funciones secundarias**

*   **calcular_inter(subset,canciones_playlist):** Recibe las tuplas de los itemsets para calcular el contador de soporte de un itemset con la lista de canciones en playlist. Una vez obtenidas las listas de reproducción en que aparece cada canción del itemset, se interseca con las listas de reproduccion de playlists. Todos los conjuntos intersecados seran el soporte de determinado itemset.

*   **combinations(iterable, r):** Este modulo sirve para generar las tuplas de combinaciones de los itemsets.

In [11]:
# Modulo para crear interseccion de los itemset
def calcular_interseccion(subset,canciones_playlist):
    playlists_interset = []
    for song in subset:
        playlists_interset.append(canciones_playlist[song])
    return len(set.intersection(*playlists_interset))
    
# Modulo para generar combinaciones en los itemsets    
def combinaciones(iterable, s):
    pool = tuple(iterable)
    # determinamos el tamaño del arreglo con len
    n = len(pool)
    if s > n:
        return
    indices = list(range(s))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(s)):
            if indices[i] != i + n - s:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, s):
            indices[j] = indices[j - 1] + 1
        yield tuple(pool[i] for i in indices)


## **2. Aplicar el algoritmo y obtener reglas de asociación**

In [17]:
# Leer datos
db = np.load("spotify.npy", allow_pickle = True)
# Generamos lositemsets
itemsets = get_frequent_itemsets(db,0.01)
itemsets[0].sort_values(by="support").head()
# Generamos las reglas
reglas_confidence,reglas_lift,reglas_ambos = generate_association_rules(itemsets,0.5,0.9)
reglas_confidence
reglas_lift
reglas_ambos.head(10)
# Top 4 de las reglas obtenidas.
top_4=reglas_lift.head(4)
top_4

,X,Y,confidence,support,lift
197,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
196,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
53,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
52,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261


## **3. Explicar las reglas obtenidas**

Se determino la filtracion de los itemset con mayor frecuencia frente a las playlist ya que se pueden encontrar coincidencias con la busqueda relacionada. 
Esto tambien varia deacuerdo al grado de certeza establecido en el umbral.


**Primera regla:** Los resultados indican que las musicas son del mismo genero esto agrega una relacion a  estos itemset y eleva el valor del umbral. 

**Segunda regla:** Cumple con lo establecido por que se repite las mismas canciones de la anterior regla y el el umbral de es mayor grado de confianza.

**Tercera regla:** Cumple con lo establecido por que la relacion que se da en esta se debe a gustos musicales y por el cual el umbral tambien es mayor.

**Cuarta regla:** Cumple con el criterio establecido.

**Nota:** De todo esto podemosdeducir que cada vez que haya una relacion este distintas reglas se debera a la relacion que ellas llevan puede ser el genero, artista, banda, etc por el cual el umbral será mayor al incial ya que cuantas mas reglas obtengamos el umbral tendrá un mayor grado de confiabilidad.